In [ ]:
# =============================================
# CÀO DỮ LIỆU BDS (TP.HCM) VÀ GHI CSV
#   pip install undetected-chromedriver
#   python bds_scraper_hcm.py
# =============================================

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, random, csv, os

# ----- CẤU HÌNH -----
PROFILE_DIR = r"C:\selenium_profiles\bds_profile_hcm"  # thư mục giữ cookie/session
os.makedirs(PROFILE_DIR, exist_ok=True)

BASE = "https://batdongsan.com.vn/nha-dat-ban-tp-hcm"  # CHỈ TP.HCM
MAX_PAGES = 4                                         # số trang muốn cào
OUTPUT_FILE = "ket_quakk.csv"                            # file kết quả
# --------------------

def build_url(page: int) -> str:
    return BASE if page == 1 else f"{BASE}/p{page}"

def init_driver():
    opts = uc.ChromeOptions()
    opts.add_argument(f"--user-data-dir={PROFILE_DIR}")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_argument("--start-maximized")
    return uc.Chrome(options=opts)

def wait_cloudflare(driver, timeout=60):
    WebDriverWait(driver, timeout).until(
        lambda d: "Verifying you are human" not in d.page_source
    )

def scroll_slow(driver):
    h = driver.execute_script("return document.body.scrollHeight")
    for y in (0.2, 0.5, 0.85):
        driver.execute_script(f"window.scrollTo(0, document.body.scrollHeight*{y});")
        time.sleep(random.uniform(0.8, 1.5))

def get_cards(driver, wait):
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".re__card-info-content")))
    return driver.find_elements(By.CSS_SELECTOR, ".re__card-info-content")

def extract_info(card):
    def safe_text(sel):
        try: return card.find_element(By.CSS_SELECTOR, sel).text.strip()
        except: return ""
    title   = safe_text(".pr-title.js__card-title")
    price   = safe_text(".re__card-config-price.js__card-config-item")
    area    = safe_text(".re__card-config-area.js__card-config-item")
    address = safe_text(".re__card-address") or safe_text(".re__card-config-location.js__card-config-item")
    date    = safe_text(".re__card-published-date") or safe_text(".re__card-config-posted")
    try:
        link = card.find_element(By.CSS_SELECTOR, "a.js__card-click").get_attribute("href") or ""
    except:
        link = ""
    return [title, price, area, address, date, link]

def main():
    driver = init_driver()
    wait = WebDriverWait(driver, 20)
    data = []

    for page in range(1, MAX_PAGES + 1):
        url = build_url(page)
        print(f"==> Trang {page}: {url}")
        driver.get(url)

        try:
            wait_cloudflare(driver, 60)
        except:
            input("Nếu Cloudflare yêu cầu xác minh, giải xong rồi nhấn ENTER để tiếp tục...")

        time.sleep(random.uniform(2, 4))
        scroll_slow(driver)

        try:
            cards = get_cards(driver, wait)
        except:
            print("⚠️ Không lấy được danh sách bài đăng, bỏ qua trang này.")
            continue

        for card in cards:
            row = extract_info(card)
            data.append(row)
            print(f"[{page}] {row[0]} | {row[1]} | {row[2]}")

        time.sleep(random.uniform(2, 5))

    # Ghi CSV (UTF-8-SIG để mở Excel tiếng Việt không lỗi)
    headers = ["Tiêu đề", "Giá", "Diện tích", "Địa chỉ", "Ngày đăng", "Link"]
    with open(OUTPUT_FILE, "w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(data)

    print(f"\n✅ Đã ghi {len(data)} dòng vào: {OUTPUT_FILE}")
    driver.quit()

if __name__ == "__main__":
    main()


full bt+bẻ khoa

In [ ]:
# =============================================
# CÀO DỮ LIỆU BDS (TP.HCM) — PHÂN TRANG + UC
# Output đúng format code1: ket_qua.csv
#   pip install undetected-chromedriver selenium
#   python bds_hcm_merged.py
# =============================================

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, random, csv, os

# ----- CẤU HÌNH -----
PROFILE_DIR = r"C:\selenium_profiles\bds_profile_hcm"   # thư mục giữ cookie/session
os.makedirs(PROFILE_DIR, exist_ok=True)

BASE = "https://batdongsan.com.vn/nha-dat-ban-tp-hcm"   # CHỈ TP.HCM
MAX_PAGES = 4                                           # số trang muốn cào: p1..pN
OUTPUT_FILE = "ket_qua.csv"                             # file kết quả (đúng tên code1)
WAIT_TIMEOUT = 20
# --------------------

def build_url(page: int) -> str:
    return BASE if page == 1 else f"{BASE}/p{page}"

def init_driver():
    opts = uc.ChromeOptions()
    opts.add_argument(f"--user-data-dir={PROFILE_DIR}")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_argument("--start-maximized")
    driver = uc.Chrome(options=opts)
    return driver

def wait_cloudflare(driver, timeout=60):
    # Chờ qua màn CF "Verifying you are human" nếu có
    WebDriverWait(driver, timeout).until(
        lambda d: "Verifying you are human" not in d.page_source
    )

def scroll_to_bottom_stable(driver, pause=1.2, max_stable=2):
    # Cuộn đến đáy cho đến khi chiều cao trang ổn định 2 lần
    last_h = driver.execute_script("return document.body.scrollHeight")
    stable = 0
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        new_h = driver.execute_script("return document.body.scrollHeight")
        if new_h == last_h:
            stable += 1
            if stable >= max_stable:
                break
        else:
            stable = 0
            last_h = new_h

def get_cards(driver, wait):
    # Lấy các card bài đăng sau khi đã cuộn
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".re__card-info-content")))
    return driver.find_elements(By.CSS_SELECTOR, ".re__card-info-content")

def safe_text_xpath(ele, xpath):
    try:
        el = ele.find_element(By.XPATH, xpath)
        return el.text.strip().replace("\n", " ")
    except:
        return "N/A"

def safe_text_css(ele, css):
    try:
        el = ele.find_element(By.CSS_SELECTOR, css)
        return el.text.strip().replace("\n", " ")
    except:
        return "N/A"

def extract_info(card):
    # Ưu tiên selector giống code1 (XPath), có fallback CSS cho ổn định
    # Tiêu đề
    tieu_de = safe_text_xpath(card, './/span[@class="pr-title js__card-title"]')
    if tieu_de == "N/A":
        tieu_de = safe_text_css(card, ".pr-title.js__card-title")
    if tieu_de == "N/A":
        tieu_de = safe_text_xpath(card, './/a[contains(@class,"js__card-title")]')

    # Giá
    gia = safe_text_xpath(card, './/span[@class="re__card-config-price js__card-config-item"]')
    if gia == "N/A":
        gia = safe_text_css(card, ".re__card-config-price.js__card-config-item")

    # Diện tích
    dien_tich = safe_text_xpath(card, './/span[@class="re__card-config-area js__card-config-item"]')
    if dien_tich == "N/A":
        dien_tich = safe_text_css(card, ".re__card-config-area.js__card-config-item")

    # Địa chỉ (loại bỏ dấu chấm giữa các phần tử nếu có)
    dia_chi = safe_text_xpath(card, './/div[contains(@class,"re__card-location")]/span[not(contains(@class,"re__card-config-dot"))]')
    if dia_chi == "N/A":
        # Fallback một số class khác nhau theo đợt cập nhật giao diện
        dia_chi = safe_text_css(card, ".re__card-address")  # hay .re__card-config-location.js__card-config-item
        if dia_chi == "N/A":
            dia_chi = safe_text_css(card, ".re__card-config-location.js__card-config-item")

    # Số phòng ngủ
    so_phong_ngu = safe_text_xpath(card, './/span[@class="re__card-config-bedroom js__card-config-item"]/span')
    if so_phong_ngu == "N/A":
        so_phong_ngu = safe_text_css(card, ".re__card-config-bedroom.js__card-config-item span")

    # Số phòng vệ sinh
    so_phong_wc = safe_text_xpath(card, './/span[@class="re__card-config-toilet js__card-config-item"]/span')
    if so_phong_wc == "N/A":
        so_phong_wc = safe_text_css(card, ".re__card-config-toilet.js__card-config-item span")

    return [tieu_de, gia, dien_tich, dia_chi, so_phong_ngu, so_phong_wc]

def main():
    driver = init_driver()
    wait = WebDriverWait(driver, WAIT_TIMEOUT)
    data = []

    for page in range(1, MAX_PAGES + 1):
        url = build_url(page)
        print(f"==> Trang {page}: {url}")
        driver.get(url)

        # Cloudflare check (nếu gặp)
        try:
            wait_cloudflare(driver, 60)
        except:
            try:
                input("Nếu Cloudflare yêu cầu xác minh, giải xong rồi nhấn ENTER để tiếp tục...")
            except EOFError:
                pass

        # Cuộn đến đáy để render hết card
        time.sleep(random.uniform(1.5, 2.5))
        scroll_to_bottom_stable(driver, pause=1.2, max_stable=2)

        # Lấy danh sách card
        try:
            cards = get_cards(driver, wait)
        except:
            print("⚠️ Không lấy được danh sách bài đăng, bỏ qua trang này.")
            continue

        print(f"Số card tìm thấy (trang {page}): {len(cards)}")

        # Trích xuất dữ liệu từng card
        for card in cards:
            row = extract_info(card)
            data.append(row)
            # In nhanh 3 cột chính khi debug
            print(f"[p{page}] {row[0]} | {row[1]} | {row[2]}")

        # Nghỉ nhẹ giữa các trang
        time.sleep(random.uniform(1.8, 3.5))

    # Ghi CSV (không có STT) theo đúng header của code1
    headers = ["Tiêu đề dự án", "Giá", "Diện tích", "Địa chỉ", "Số phòng ngủ", "Số phòng vệ sinh"]
    csv_path = os.path.join(os.getcwd(), OUTPUT_FILE)
    print("Sẽ ghi vào:", csv_path)

    with open(csv_path, "w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(data)

    print(f"\n✅ Đã lưu {len(data)} dòng vào file: {csv_path}")
    driver.quit()

if __name__ == "__main__":
    main()


full + be khoa ko hien teminal

In [ ]:
# =============================================
# CÀO DỮ LIỆU BDS (TP.HCM) — PHÂN TRANG + UC
# Output: ket_qua.csv (không in từng dòng)
# =============================================

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, random, csv, os

# ----- CẤU HÌNH -----
PROFILE_DIR = r"C:\selenium_profiles\bds_profile_hcm"
os.makedirs(PROFILE_DIR, exist_ok=True)

BASE = "https://batdongsan.com.vn/nha-dat-ban-tp-hcm"
MAX_PAGES = 2
OUTPUT_FILE = "ket_quasđ.csv"
WAIT_TIMEOUT = 20
# --------------------

def build_url(page: int) -> str:
    return BASE if page == 1 else f"{BASE}/p{page}"

def init_driver():
    opts = uc.ChromeOptions()
    opts.add_argument(f"--user-data-dir={PROFILE_DIR}")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_argument("--start-maximized")
    return uc.Chrome(options=opts)

def wait_cloudflare(driver, timeout=60):
    WebDriverWait(driver, timeout).until(
        lambda d: "Verifying you are human" not in d.page_source
    )

def scroll_to_bottom_stable(driver, pause=1.2, max_stable=2):
    last_h = driver.execute_script("return document.body.scrollHeight")
    stable = 0
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        new_h = driver.execute_script("return document.body.scrollHeight")
        if new_h == last_h:
            stable += 1
            if stable >= max_stable:
                break
        else:
            stable = 0
            last_h = new_h

def get_cards(driver, wait):
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".re__card-info-content")))
    return driver.find_elements(By.CSS_SELECTOR, ".re__card-info-content")

def safe_text_xpath(ele, xpath):
    try:
        el = ele.find_element(By.XPATH, xpath)
        return el.text.strip().replace("\n", " ")
    except:
        return "N/A"

def safe_text_css(ele, css):
    try:
        el = ele.find_element(By.CSS_SELECTOR, css)
        return el.text.strip().replace("\n", " ")
    except:
        return "N/A"

def extract_info(card):
    tieu_de = safe_text_xpath(card, './/span[@class="pr-title js__card-title"]')
    if tieu_de == "N/A":
        tieu_de = safe_text_css(card, ".pr-title.js__card-title")
    if tieu_de == "N/A":
        tieu_de = safe_text_xpath(card, './/a[contains(@class,"js__card-title")]')

    gia = safe_text_xpath(card, './/span[@class="re__card-config-price js__card-config-item"]')
    if gia == "N/A":
        gia = safe_text_css(card, ".re__card-config-price.js__card-config-item")

    dien_tich = safe_text_xpath(card, './/span[@class="re__card-config-area js__card-config-item"]')
    if dien_tich == "N/A":
        dien_tich = safe_text_css(card, ".re__card-config-area.js__card-config-item")

    dia_chi = safe_text_xpath(card, './/div[contains(@class,"re__card-location")]/span[not(contains(@class,"re__card-config-dot"))]')
    if dia_chi == "N/A":
        dia_chi = safe_text_css(card, ".re__card-address")
        if dia_chi == "N/A":
            dia_chi = safe_text_css(card, ".re__card-config-location.js__card-config-item")

    so_phong_ngu = safe_text_xpath(card, './/span[@class="re__card-config-bedroom js__card-config-item"]/span')
    if so_phong_ngu == "N/A":
        so_phong_ngu = safe_text_css(card, ".re__card-config-bedroom.js__card-config-item span")

    so_phong_wc = safe_text_xpath(card, './/span[@class="re__card-config-toilet js__card-config-item"]/span')
    if so_phong_wc == "N/A":
        so_phong_wc = safe_text_css(card, ".re__card-config-toilet.js__card-config-item span")

    return [tieu_de, gia, dien_tich, dia_chi, so_phong_ngu, so_phong_wc]

def main():
    driver = init_driver()
    wait = WebDriverWait(driver, WAIT_TIMEOUT)
    data = []

    for page in range(1, MAX_PAGES + 1):
        url = build_url(page)
        print(f"==> Đang xử lý trang {page}: {url}")
        driver.get(url)

        try:
            wait_cloudflare(driver, 60)
        except:
            try:
                input("Nếu Cloudflare yêu cầu xác minh, giải xong rồi nhấn ENTER để tiếp tục...")
            except EOFError:
                pass

        time.sleep(random.uniform(1.5, 2.5))
        scroll_to_bottom_stable(driver, pause=1.2, max_stable=2)

        try:
            cards = get_cards(driver, wait)
        except:
            print("⚠️ Không lấy được danh sách bài đăng, bỏ qua trang này.")
            continue

        print(f"   ↳ Số bài đăng tìm thấy: {len(cards)}")

        for card in cards:
            row = extract_info(card)
            data.append(row)

        # Hiện tiến độ nhẹ nhàng
        print(f"   ✅ Đã thu thập tổng cộng {len(data)} dòng\n")
        time.sleep(random.uniform(1.8, 3.5))

    headers = ["Tiêu đề dự án", "Giá", "Diện tích", "Địa chỉ", "Số phòng ngủ", "Số phòng vệ sinh"]
    csv_path = os.path.join(os.getcwd(), OUTPUT_FILE)
    print("Sẽ ghi vào:", csv_path)

    with open(csv_path, "w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(data)

    print(f"\n✅ Đã lưu {len(data)} dòng vào file: {csv_path}")
    driver.quit()

if __name__ == "__main__":
    main()


full + be khoa + crawwl full ko max page(hoan thien)

In [ ]:
# =============================================
# BDS TP.HCM — Cào "FULL" đến khi hết trang
# - Output: ket_qua.csv (đúng format code1)
# - Ghi từng dòng (không giữ toàn bộ vào RAM)
# - Tự dừng khi trang không còn card
# - Có checkpoint để resume
# =============================================

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, random, csv, os, sys

# ===== CẤU HÌNH =====
PROFILE_DIR  = r"C:\selenium_profiles\bds_profile_hcm"
BASE         = "https://batdongsan.com.vn/nha-dat-ban-tp-hcm"
OUTPUT_FILE  = "ket_qua1.csv"
CHECKPOINT   = "bds_checkpoint.txt"   # lưu trang gần nhất đã xong
WAIT_TIMEOUT = 20
SAFETY_CAP   = 10000                  # giới hạn an toàn số trang (tránh loop vô hạn)
HEADLESS     = False                  # True nếu muốn chạy ẩn
# ====================

os.makedirs(PROFILE_DIR, exist_ok=True)

def build_url(page: int) -> str:
    return BASE if page == 1 else f"{BASE}/p{page}"

def init_driver():
    opts = uc.ChromeOptions()
    opts.add_argument(f"--user-data-dir={PROFILE_DIR}")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_argument("--start-maximized")
    if HEADLESS:
        opts.add_argument("--headless=new")
    return uc.Chrome(options=opts)

def wait_cloudflare(driver, timeout=60):
    WebDriverWait(driver, timeout).until(
        lambda d: "Verifying you are human" not in d.page_source
    )

def scroll_to_bottom_stable(driver, pause=1.1, max_stable=2):
    last_h = driver.execute_script("return document.body.scrollHeight")
    stable = 0
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        new_h = driver.execute_script("return document.body.scrollHeight")
        if new_h == last_h:
            stable += 1
            if stable >= max_stable:
                break
        else:
            stable = 0
            last_h = new_h

def get_cards(driver, wait):
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".re__card-info-content")))
    return driver.find_elements(By.CSS_SELECTOR, ".re__card-info-content")

def safe_text_xpath(ele, xpath):
    try:
        el = ele.find_element(By.XPATH, xpath)
        return el.text.strip().replace("\n", " ")
    except:
        return "N/A"

def safe_text_css(ele, css):
    try:
        el = ele.find_element(By.CSS_SELECTOR, css)
        return el.text.strip().replace("\n", " ")
    except:
        return "N/A"

def extract_info(card):
    tieu_de = safe_text_xpath(card, './/span[@class="pr-title js__card-title"]')
    if tieu_de == "N/A":
        tieu_de = safe_text_css(card, ".pr-title.js__card-title")
    if tieu_de == "N/A":
        tieu_de = safe_text_xpath(card, './/a[contains(@class,"js__card-title")]')

    gia = safe_text_xpath(card, './/span[@class="re__card-config-price js__card-config-item"]')
    if gia == "N/A":
        gia = safe_text_css(card, ".re__card-config-price.js__card-config-item")

    dien_tich = safe_text_xpath(card, './/span[@class="re__card-config-area js__card-config-item"]')
    if dien_tich == "N/A":
        dien_tich = safe_text_css(card, ".re__card-config-area.js__card-config-item")

    dia_chi = safe_text_xpath(card, './/div[contains(@class,"re__card-location")]/span[not(contains(@class,"re__card-config-dot"))]')
    if dia_chi == "N/A":
        dia_chi = safe_text_css(card, ".re__card-address")
        if dia_chi == "N/A":
            dia_chi = safe_text_css(card, ".re__card-config-location.js__card-config-item")

    so_phong_ngu = safe_text_xpath(card, './/span[@class="re__card-config-bedroom js__card-config-item"]/span')
    if so_phong_ngu == "N/A":
        so_phong_ngu = safe_text_css(card, ".re__card-config-bedroom.js__card-config-item span")

    so_phong_wc = safe_text_xpath(card, './/span[@class="re__card-config-toilet js__card-config-item"]/span')
    if so_phong_wc == "N/A":
        so_phong_wc = safe_text_css(card, ".re__card-config-toilet.js__card-config-item span")

    return [tieu_de, gia, dien_tich, dia_chi, so_phong_ngu, so_phong_wc]

def read_checkpoint() -> int:
    if os.path.exists(CHECKPOINT):
        try:
            with open(CHECKPOINT, "r", encoding="utf-8") as f:
                p = int(f.read().strip())
                return max(1, p)
        except:
            return 1
    return 1

def write_checkpoint(page: int):
    with open(CHECKPOINT, "w", encoding="utf-8") as f:
        f.write(str(page))

def ensure_csv_header(path: str):
    need_header = not os.path.exists(path) or os.path.getsize(path) == 0
    if need_header:
        with open(path, "w", encoding="utf-8-sig", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["Tiêu đề dự án", "Giá", "Diện tích", "Địa chỉ", "Số phòng ngủ", "Số phòng vệ sinh"])

def main():
    driver = init_driver()
    wait = WebDriverWait(driver, WAIT_TIMEOUT)
    csv_path = os.path.join(os.getcwd(), OUTPUT_FILE)
    ensure_csv_header(csv_path)

    # bắt đầu từ checkpoint
    page = read_checkpoint()
    processed_rows = 0

    while page <= SAFETY_CAP:
        url = build_url(page)
        print(f"==> Trang {page}: {url}")
        driver.get(url)

        # Cloudflare
        try:
            wait_cloudflare(driver, 60)
        except:
            try:
                input("Nếu Cloudflare yêu cầu xác minh, giải xong rồi nhấn ENTER để tiếp tục...")
            except EOFError:
                pass

        # cuộn để load đủ card
        time.sleep(random.uniform(1.5, 2.5))
        scroll_to_bottom_stable(driver, pause=1.0, max_stable=2)

        # lấy card; nếu không có -> coi như hết trang => dừng
        try:
            cards = get_cards(driver, wait)
        except:
            cards = []
        if not cards:
            print("   ↳ Hết dữ liệu (không còn card). Dừng.")
            break

        print(f"   ↳ Số bài đăng: {len(cards)}")

        # ghi từng dòng ngay lập tức
        with open(csv_path, "a", encoding="utf-8-sig", newline="") as f:
            writer = csv.writer(f)
            for c in cards:
                row = extract_info(c)
                writer.writerow(row)
                processed_rows += 1

        print(f"   ✅ Tổng đã ghi: {processed_rows} dòng\n")

        # lưu checkpoint trang vừa xong và chuyển tiếp
        write_checkpoint(page)
        page += 1
        time.sleep(random.uniform(1.6, 3.2))

    print(f"✅ Hoàn tất. Đã ghi {processed_rows} dòng vào: {csv_path}")
    driver.quit()

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n⏸ Dừng bởi người dùng. Có thể chạy lại, chương trình sẽ tiếp tục từ checkpoint.")
        sys.exit(0)


==> Trang 1: https://batdongsan.com.vn/nha-dat-ban-tp-hcm
   ↳ Số bài đăng: 29
   ✅ Tổng đã ghi: 29 dòng

==> Trang 2: https://batdongsan.com.vn/nha-dat-ban-tp-hcm/p2
   ↳ Số bài đăng: 29
   ✅ Tổng đã ghi: 58 dòng

==> Trang 3: https://batdongsan.com.vn/nha-dat-ban-tp-hcm/p3


In [ ]:
test

In [1]:
# =============================================
# BDS TP.HCM — Cào "FULL" đến khi hết trang
# - Output: ket_qua.csv (thứ tự cột đúng như hiển thị trên web)
# - Ghi từng dòng (không giữ toàn bộ vào RAM)
# - Tự dừng khi trang không còn card
# - Có checkpoint để resume
# =============================================

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, random, csv, os, sys

# ===== CẤU HÌNH =====
PROFILE_DIR  = r"C:\selenium_profiles\bds_profile_hcm"
BASE         = "https://batdongsan.com.vn/nha-dat-ban-tp-hcm"
OUTPUT_FILE  = "ket_quaokok.csv"
CHECKPOINT   = "bds_checkpoint.txt"
WAIT_TIMEOUT = 20
SAFETY_CAP   = 10000
HEADLESS     = False
# ====================

os.makedirs(PROFILE_DIR, exist_ok=True)

def build_url(page: int) -> str:
    return BASE if page == 1 else f"{BASE}/p{page}"

def init_driver():
    opts = uc.ChromeOptions()
    opts.add_argument(f"--user-data-dir={PROFILE_DIR}")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_argument("--start-maximized")
    if HEADLESS:
        opts.add_argument("--headless=new")
    return uc.Chrome(options=opts)

def wait_cloudflare(driver, timeout=60):
    WebDriverWait(driver, timeout).until(
        lambda d: "Verifying you are human" not in d.page_source
    )

def scroll_to_bottom_stable(driver, pause=1.1, max_stable=2):
    last_h = driver.execute_script("return document.body.scrollHeight")
    stable = 0
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        new_h = driver.execute_script("return document.body.scrollHeight")
        if new_h == last_h:
            stable += 1
            if stable >= max_stable:
                break
        else:
            stable = 0
            last_h = new_h

def get_cards(driver, wait):
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".re__card-info-content")))
    return driver.find_elements(By.CSS_SELECTOR, ".re__card-info-content")

def safe_text_xpath(ele, xpath):
    try:
        el = ele.find_element(By.XPATH, xpath)
        return el.text.strip().replace("\n", " ")
    except:
        return "N/A"

def safe_text_css(ele, css):
    try:
        el = ele.find_element(By.CSS_SELECTOR, css)
        return el.text.strip().replace("\n", " ")
    except:
        return "N/A"

def extract_gia_m2(card):
    """Lấy giá/m² nếu có hiển thị như '200 tr/m²'."""
    try:
        spans = card.find_elements(By.XPATH, './/span[contains(@class,"js__card-config-item")]')
        for s in spans:
            t = s.text.lower().strip()
            if "/m2" in t or "/m²" in t:
                return s.text.strip()
    except:
        pass
    return "N/A"

def extract_info(card):
    # ---- Tiêu đề ----
    tieu_de = safe_text_xpath(card, './/span[@class="pr-title js__card-title"]')
    if tieu_de == "N/A":
        tieu_de = safe_text_css(card, ".pr-title.js__card-title")
    if tieu_de == "N/A":
        tieu_de = safe_text_xpath(card, './/a[contains(@class,"js__card-title")]')

    # ---- Giá ----
    gia = safe_text_xpath(card, './/span[@class="re__card-config-price js__card-config-item"]')
    if gia == "N/A":
        gia = safe_text_css(card, ".re__card-config-price.js__card-config-item")

    # ---- Diện tích ----
    dien_tich = safe_text_xpath(card, './/span[@class="re__card-config-area js__card-config-item"]')
    if dien_tich == "N/A":
        dien_tich = safe_text_css(card, ".re__card-config-area.js__card-config-item")

    # ---- Giá/m² ----
    gia_m2 = extract_gia_m2(card)

    # ---- Địa chỉ ----
    dia_chi = safe_text_xpath(card, './/div[contains(@class,"re__card-location")]/span[not(contains(@class,"re__card-config-dot"))]')
    if dia_chi == "N/A":
        dia_chi = safe_text_css(card, ".re__card-address")
        if dia_chi == "N/A":
            dia_chi = safe_text_css(card, ".re__card-config-location.js__card-config-item")

    # ---- Số phòng ngủ ----
    so_phong_ngu = safe_text_xpath(card, './/span[@class="re__card-config-bedroom js__card-config-item"]/span')
    if so_phong_ngu == "N/A":
        so_phong_ngu = safe_text_css(card, ".re__card-config-bedroom.js__card-config-item span")

    # ---- Số phòng vệ sinh ----
    so_phong_wc = safe_text_xpath(card, './/span[@class="re__card-config-toilet js__card-config-item"]/span')
    if so_phong_wc == "N/A":
        so_phong_wc = safe_text_css(card, ".re__card-config-toilet.js__card-config-item span")

    # Thứ tự theo hiển thị trên web
    return [tieu_de, gia, dien_tich, gia_m2, dia_chi, so_phong_ngu, so_phong_wc]

def read_checkpoint() -> int:
    if os.path.exists(CHECKPOINT):
        try:
            with open(CHECKPOINT, "r", encoding="utf-8") as f:
                return max(1, int(f.read().strip()))
        except:
            return 1
    return 1

def write_checkpoint(page: int):
    with open(CHECKPOINT, "w", encoding="utf-8") as f:
        f.write(str(page))

def ensure_csv_header(path: str):
    if not os.path.exists(path) or os.path.getsize(path) == 0:
        with open(path, "w", encoding="utf-8-sig", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([
                "Tiêu đề dự án",
                "Giá",
                "Diện tích",
                "Giá/m²",
                "Địa chỉ",
                "Số phòng ngủ",
                "Số phòng vệ sinh"
            ])

def main():
    driver = init_driver()
    wait = WebDriverWait(driver, WAIT_TIMEOUT)
    csv_path = os.path.join(os.getcwd(), OUTPUT_FILE)
    ensure_csv_header(csv_path)

    page = read_checkpoint()
    processed = 0

    while page <= SAFETY_CAP:
        url = build_url(page)
        print(f"==> Trang {page}: {url}")
        driver.get(url)

        try:
            wait_cloudflare(driver, 60)
        except:
            input("Nếu Cloudflare yêu cầu xác minh, giải xong rồi nhấn ENTER để tiếp tục...")

        time.sleep(random.uniform(1.5, 2.5))
        scroll_to_bottom_stable(driver, pause=1.0, max_stable=2)

        try:
            cards = get_cards(driver, wait)
        except:
            cards = []
        if not cards:
            print("↳ Hết dữ liệu (không còn card). Dừng.")
            break

        print(f"↳ Số bài đăng: {len(cards)}")

        with open(csv_path, "a", encoding="utf-8-sig", newline="") as f:
            w = csv.writer(f)
            for c in cards:
                w.writerow(extract_info(c))
                processed += 1

        write_checkpoint(page)
        page += 1
        print(f"✅ Tổng đã ghi: {processed} dòng\n")
        time.sleep(random.uniform(1.6, 3.0))

    print(f"✅ Hoàn tất. Đã ghi {processed} dòng vào {csv_path}")
    driver.quit()

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n⏸ Dừng bởi người dùng. Có thể chạy lại từ checkpoint.")
        sys.exit(0)



==> Trang 1: https://batdongsan.com.vn/nha-dat-ban-tp-hcm
↳ Số bài đăng: 29
✅ Tổng đã ghi: 29 dòng

==> Trang 2: https://batdongsan.com.vn/nha-dat-ban-tp-hcm/p2
↳ Số bài đăng: 29
✅ Tổng đã ghi: 58 dòng

==> Trang 3: https://batdongsan.com.vn/nha-dat-ban-tp-hcm/p3

⏸ Dừng bởi người dùng. Có thể chạy lại từ checkpoint.


SystemExit: 0

c:\Users\ADMIN\miniconda3\envs\mconda311\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


hoan thien(ch co ngay dang)

In [2]:
# =============================================
# BDS TP.HCM — Cào "FULL" đến khi hết trang
# - Output: ket_quaokok.csv (thứ tự cột đúng như hiển thị trên web)
# - Ghi từng dòng (không giữ toàn bộ vào RAM)
# - Tự dừng khi trang không còn card
# - Có checkpoint để resume
# - ĐÃ THÊM: Lọc bỏ quảng cáo/thẻ rác (không tiêu đề/không giá)
# =============================================

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, random, csv, os, sys

# ===== CẤU HÌNH =====
PROFILE_DIR  = r"C:\selenium_profiles\bds_profile_hcm"
BASE         = "https://batdongsan.com.vn/nha-dat-ban-tp-hcm"
OUTPUT_FILE  = "ket_quaokok.csv"
CHECKPOINT   = "bds_checkpoint.txt"
WAIT_TIMEOUT = 20
SAFETY_CAP   = 10000
HEADLESS     = False
# ====================

os.makedirs(PROFILE_DIR, exist_ok=True)

def build_url(page: int) -> str:
    return BASE if page == 1 else f"{BASE}/p{page}"

def init_driver():
    opts = uc.ChromeOptions()
    opts.add_argument(f"--user-data-dir={PROFILE_DIR}")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_argument("--start-maximized")
    # Ổn định hơn với lazy-load
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--no-sandbox")
    if HEADLESS:
        opts.add_argument("--headless=new")
    return uc.Chrome(options=opts)

def wait_cloudflare(driver, timeout=60):
    WebDriverWait(driver, timeout).until(
        lambda d: "Verifying you are human" not in d.page_source
    )

def scroll_to_bottom_stable(driver, pause=1.1, max_stable=2):
    last_h = driver.execute_script("return document.body.scrollHeight")
    stable = 0
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        new_h = driver.execute_script("return document.body.scrollHeight")
        if new_h == last_h:
            stable += 1
            if stable >= max_stable:
                break
        else:
            stable = 0
            last_h = new_h

def safe_text_xpath(ele, xpath):
    try:
        el = ele.find_element(By.XPATH, xpath)
        return el.text.strip().replace("\n", " ")
    except:
        return "N/A"

def safe_text_css(ele, css):
    try:
        el = ele.find_element(By.CSS_SELECTOR, css)
        return el.text.strip().replace("\n", " ")
    except:
        return "N/A"

def extract_gia_m2(card):
    """Lấy giá/m² nếu có hiển thị như '200 tr/m²'."""
    try:
        spans = card.find_elements(By.XPATH, './/span[contains(@class,"js__card-config-item")]')
        for s in spans:
            t = s.text.lower().strip()
            if "/m2" in t or "/m²" in t:
                return s.text.strip()
    except:
        pass
    return "N/A"

def extract_info(card):
    # ---- Tiêu đề ----
    tieu_de = safe_text_xpath(card, './/span[@class="pr-title js__card-title"]')
    if tieu_de == "N/A":
        tieu_de = safe_text_css(card, ".pr-title.js__card-title")
    if tieu_de == "N/A":
        tieu_de = safe_text_xpath(card, './/a[contains(@class,"js__card-title")]')

    # ---- Giá ----
    gia = safe_text_xpath(card, './/span[@class="re__card-config-price js__card-config-item"]')
    if gia == "N/A":
        gia = safe_text_css(card, ".re__card-config-price.js__card-config-item")

    # ---- Diện tích ----
    dien_tich = safe_text_xpath(card, './/span[@class="re__card-config-area js__card-config-item"]')
    if dien_tich == "N/A":
        dien_tich = safe_text_css(card, ".re__card-config-area.js__card-config-item")

    # ---- Giá/m² ----
    gia_m2 = extract_gia_m2(card)

    # ---- Địa chỉ ----
    dia_chi = safe_text_xpath(card, './/div[contains(@class,"re__card-location")]/span[not(contains(@class,"re__card-config-dot"))]')
    if dia_chi == "N/A":
        dia_chi = safe_text_css(card, ".re__card-address")
        if dia_chi == "N/A":
            dia_chi = safe_text_css(card, ".re__card-config-location.js__card-config-item")

    # ---- Số phòng ngủ ----
    so_phong_ngu = safe_text_xpath(card, './/span[@class="re__card-config-bedroom js__card-config-item"]/span')
    if so_phong_ngu == "N/A":
        so_phong_ngu = safe_text_css(card, ".re__card-config-bedroom.js__card-config-item span")

    # ---- Số phòng vệ sinh ----
    so_phong_wc = safe_text_xpath(card, './/span[@class="re__card-config-toilet js__card-config-item"]/span')
    if so_phong_wc == "N/A":
        so_phong_wc = safe_text_css(card, ".re__card-config-toilet.js__card-config-item span")

    # Thứ tự theo hiển thị trên web
    return [tieu_de, gia, dien_tich, gia_m2, dia_chi, so_phong_ngu, so_phong_wc]

# ---------- LỌC QUẢNG CÁO / RÁC ----------
def get_cards(driver, wait):
    """Trả về chỉ các card hợp lệ (có tiêu đề & giá)."""
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".re__card-info-content")))
    all_cards = driver.find_elements(By.CSS_SELECTOR, ".re__card-info-content")
    valid_cards = []

    for card in all_cards:
        try:
            title = safe_text_xpath(card, './/span[@class="pr-title js__card-title"]')
            if title == "N/A":
                title = safe_text_css(card, ".pr-title.js__card-title")
            price = safe_text_xpath(card, './/span[@class="re__card-config-price js__card-config-item"]')
            if price == "N/A":
                price = safe_text_css(card, ".re__card-config-price.js__card-config-item")

            # Điều kiện loại bỏ quảng cáo/thẻ rỗng
            if not title or title == "N/A" or title.strip() == "":
                continue
            if not price or price == "N/A" or price.strip() == "":
                continue

            # Một số banner có icon đặc thù → nếu muốn: bỏ luôn
            # try:
            #     card.find_element(By.CSS_SELECTOR, ".re__card-advertisement, .ads, .sponsor")
            #     continue  # nếu có class quảng cáo thì bỏ
            # except:
            #     pass

            valid_cards.append(card)
        except:
            continue
    return valid_cards

def read_checkpoint() -> int:
    if os.path.exists(CHECKPOINT):
        try:
            with open(CHECKPOINT, "r", encoding="utf-8") as f:
                return max(1, int(f.read().strip()))
        except:
            return 1
    return 1

def write_checkpoint(page: int):
    with open(CHECKPOINT, "w", encoding="utf-8") as f:
        f.write(str(page))

def ensure_csv_header(path: str):
    if not os.path.exists(path) or os.path.getsize(path) == 0:
        with open(path, "w", encoding="utf-8-sig", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([
                "Tiêu đề dự án",
                "Giá",
                "Diện tích",
                "Giá/m²",
                "Địa chỉ",
                "Số phòng ngủ",
                "Số phòng vệ sinh"
            ])

def main():
    driver = init_driver()
    wait = WebDriverWait(driver, WAIT_TIMEOUT)
    csv_path = os.path.join(os.getcwd(), OUTPUT_FILE)
    ensure_csv_header(csv_path)

    page = read_checkpoint()
    processed = 0

    while page <= SAFETY_CAP:
        url = build_url(page)
        print(f"==> Trang {page}: {url}")
        driver.get(url)

        try:
            wait_cloudflare(driver, 60)
        except:
            input("Nếu Cloudflare yêu cầu xác minh, giải xong rồi nhấn ENTER để tiếp tục...")

        time.sleep(random.uniform(1.5, 2.5))
        scroll_to_bottom_stable(driver, pause=1.0, max_stable=2)

        try:
            cards = get_cards(driver, wait)
        except:
            cards = []

        # Nếu sau khi lọc không còn card hợp lệ → coi như hết dữ liệu
        if not cards:
            print("↳ Hết dữ liệu (không còn card hợp lệ). Dừng.")
            break

        print(f"↳ Số bài đăng hợp lệ: {len(cards)}")

        with open(csv_path, "a", encoding="utf-8-sig", newline="") as f:
            w = csv.writer(f)
            for c in cards:
                row = extract_info(c)
                # Bỏ dòng toàn N/A cho chắc
                if all(x == "N/A" for x in row):
                    continue
                w.writerow(row)
                processed += 1

        write_checkpoint(page)
        page += 1
        print(f"✅ Tổng đã ghi: {processed} dòng\n")
        time.sleep(random.uniform(1.6, 3.0))

    print(f"✅ Hoàn tất. Đã ghi {processed} dòng vào {csv_path}")
    driver.quit()

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n⏸ Dừng bởi người dùng. Có thể chạy lại từ checkpoint.")
        sys.exit(0)




⏸ Dừng bởi người dùng. Có thể chạy lại từ checkpoint.


SystemExit: 0

c:\Users\ADMIN\miniconda3\envs\mconda311\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


hoan thien(nhung cao lau vi loi chome driver)

In [2]:
# =============================================
# BDS TP.HCM — Cào "FULL" đến khi hết trang
# - Output: ket_quaokok.csv (thứ tự cột đúng như hiển thị trên web)
# - Ghi từng dòng (không giữ toàn bộ vào RAM)
# - Tự dừng khi trang không còn card
# - Có checkpoint để resume
# - Lọc bỏ quảng cáo/thẻ rác (không tiêu đề/không giá)
# - ĐÃ THÊM: Trường "Ngày đăng" (lấy theo aria-label/text)
# - ĐÃ FIX: Khởi tạo UC ổn định (retry + tránh kẹt profile), ép Chrome major
# =============================================

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException, TimeoutException
import time, random, csv, os, sys, tempfile, shutil

# ===== CẤU HÌNH =====
PROFILE_DIR   = r"C:\selenium_profiles\bds_profile_hcm"  # nếu muốn dùng profile cố định
BASE          = "https://batdongsan.com.vn/nha-dat-ban-tp-hcm"
OUTPUT_FILE   = "ketqua_rawdl.csv"
CHECKPOINT    = "bds_checkpoint.txt"
WAIT_TIMEOUT  = 20
SAFETY_CAP    = 10000
HEADLESS      = False

# Ép khớp major version của Chrome (mở chrome://version để xem, vd 141.x.y):
CHROME_MAJOR  = 141

# Dùng profile tạm để tránh kẹt (đề xuất True). Nếu bạn cần giữ login/CF, đặt False để dùng PROFILE_DIR.
USE_TEMP_PROFILE = True

# ====================
os.makedirs(PROFILE_DIR, exist_ok=True)

def build_url(page: int) -> str:
    return BASE if page == 1 else f"{BASE}/p{page}"

def _make_options(tmp_profile_dir: str | None):
    opts = uc.ChromeOptions()
    if tmp_profile_dir:
        opts.add_argument(f"--user-data-dir={tmp_profile_dir}")
    else:
        opts.add_argument(f"--user-data-dir={PROFILE_DIR}")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_argument("--start-maximized")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--no-sandbox")
    if HEADLESS:
        opts.add_argument("--headless=new")
    return opts

def init_driver():
    """
    Khởi tạo undetected_chromedriver an toàn:
    - Tạo profile tạm (mặc định) để tránh khoá/lock
    - Retry 3 lần nếu lỗi SessionNotCreatedException
    - Ép CHROME_MAJOR cho khớp Chrome
    """
    tmp_profile = tempfile.mkdtemp(prefix="bds_uc_") if USE_TEMP_PROFILE else None
    opts = _make_options(tmp_profile)

    last_err = None
    for attempt in range(1, 4):
        try:
            drv = uc.Chrome(
                options=opts,
                version_main=CHROME_MAJOR,
                use_subprocess=True,
                suppress_welcome=True,
            )
            drv._tmp_profile_dir = tmp_profile
            return drv
        except WebDriverException as e:
            last_err = e
            time.sleep(2 * attempt)

    # thất bại: dọn profile tạm nếu có rồi ném lỗi
    if tmp_profile:
        shutil.rmtree(tmp_profile, ignore_errors=True)
    raise last_err

def safe_quit(driver):
    try:
        driver.quit()
    except:
        pass
    # dọn profile tạm nếu có
    tmp = getattr(driver, "_tmp_profile_dir", None)
    if tmp and os.path.isdir(tmp):
        shutil.rmtree(tmp, ignore_errors=True)

def wait_cloudflare(driver, timeout=60):
    WebDriverWait(driver, timeout).until(
        lambda d: "Verifying you are human" not in d.page_source
    )

def scroll_to_bottom_stable(driver, pause=1.1, max_stable=2):
    last_h = driver.execute_script("return document.body.scrollHeight")
    stable = 0
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        new_h = driver.execute_script("return document.body.scrollHeight")
        if new_h == last_h:
            stable += 1
            if stable >= max_stable:
                break
        else:
            stable = 0
            last_h = new_h

def safe_text_xpath(ele, xpath):
    try:
        el = ele.find_element(By.XPATH, xpath)
        return el.text.strip().replace("\n", " ")
    except:
        return "N/A"

def safe_text_css(ele, css):
    try:
        el = ele.find_element(By.CSS_SELECTOR, css)
        return el.text.strip().replace("\n", " ")
    except:
        return "N/A"

def container_of(ele):
    """Tìm thẻ bao ngoài của card (để lấy 'Ngày đăng' vốn đôi khi nằm ngoài .re__card-info-content)."""
    try:
        return ele.find_element(
            By.XPATH,
            "./ancestor::*[contains(@class,'re__srp-card') or contains(@class,'js_card') or contains(@class,'re__card')][1]"
        )
    except:
        return ele

def get_ngay_dang(root):
    """Ưu tiên lấy từ aria-label; fallback text."""
    for sel in [
        "span.re__card-published-info-published-at",
        ".re__card-published-info [aria-label]",
        ".agent-listing-time [aria-label]",
        ".re__card-config-time",  # thêm 1 case hay gặp
    ]:
        try:
            el = root.find_element(By.CSS_SELECTOR, sel)
            val = (el.get_attribute("aria-label") or el.text or "").strip()
            if val:
                return val
        except:
            pass
    return "N/A"

def extract_gia_m2(card):
    """Lấy giá/m² nếu có hiển thị như '200 tr/m²'."""
    try:
        spans = card.find_elements(By.XPATH, './/span[contains(@class,"js__card-config-item")]')
        for s in spans:
            t = s.text.lower().strip()
            if "/m2" in t or "/m²" in t:
                return s.text.strip()
    except:
        pass
    return "N/A"

def extract_info(card):
    # ---- Tiêu đề ----
    tieu_de = safe_text_xpath(card, './/span[@class="pr-title js__card-title"]')
    if tieu_de == "N/A":
        tieu_de = safe_text_css(card, ".pr-title.js__card-title")
    if tieu_de == "N/A":
        tieu_de = safe_text_xpath(card, './/a[contains(@class,"js__card-title")]')

    # ---- Giá ----
    gia = safe_text_xpath(card, './/span[@class="re__card-config-price js__card-config-item"]')
    if gia == "N/A":
        gia = safe_text_css(card, ".re__card-config-price.js__card-config-item")

    # ---- Diện tích ----
    dien_tich = safe_text_xpath(card, './/span[@class="re__card-config-area js__card-config-item"]')
    if dien_tich == "N/A":
        dien_tich = safe_text_css(card, ".re__card-config-area.js__card-config-item")

    # ---- Giá/m² ----
    gia_m2 = extract_gia_m2(card)

    # ---- Địa chỉ ----
    dia_chi = safe_text_xpath(
        card, './/div[contains(@class,"re__card-location")]/span[not(contains(@class,"re__card-config-dot"))]'
    )
    if dia_chi == "N/A":
        dia_chi = safe_text_css(card, ".re__card-address")
    if dia_chi == "N/A":
        dia_chi = safe_text_css(card, ".re__card-config-location.js__card-config-item")

    # ---- Số phòng ngủ ----
    so_phong_ngu = safe_text_xpath(card, './/span[@class="re__card-config-bedroom js__card-config-item"]/span')
    if so_phong_ngu == "N/A":
        so_phong_ngu = safe_text_css(card, ".re__card-config-bedroom.js__card-config-item span")

    # ---- Số phòng vệ sinh ----
    so_phong_wc = safe_text_xpath(card, './/span[@class="re__card-config-toilet js__card-config-item"]/span')
    if so_phong_wc == "N/A":
        so_phong_wc = safe_text_css(card, ".re__card-config-toilet.js__card-config-item span")

    # ---- Ngày đăng (lấy trên container ngoài) ----
    root = container_of(card)
    ngay_dang = get_ngay_dang(root)

    # Thứ tự theo hiển thị + "Ngày đăng" (cuối cùng)
    return [tieu_de, gia, dien_tich, gia_m2, dia_chi, so_phong_ngu, so_phong_wc, ngay_dang]

# ---------- LỌC QUẢNG CÁO / RÁC ----------
def get_cards(driver, wait):
    """Trả về chỉ các card hợp lệ (có tiêu đề & giá)."""
    try:
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".re__card-info-content")))
    except TimeoutException:
        return []

    all_cards = driver.find_elements(By.CSS_SELECTOR, ".re__card-info-content")
    valid_cards = []
    for card in all_cards:
        try:
            title = safe_text_xpath(card, './/span[@class="pr-title js__card-title"]')
            if title == "N/A":
                title = safe_text_css(card, ".pr-title.js__card-title")

            price = safe_text_xpath(card, './/span[@class="re__card-config-price js__card-config-item"]')
            if price == "N/A":
                price = safe_text_css(card, ".re__card-config-price.js__card-config-item")

            # Điều kiện loại bỏ quảng cáo/thẻ rỗng
            if not title or title == "N/A" or not title.strip():
                continue
            if not price or price == "N/A" or not price.strip():
                continue

            valid_cards.append(card)
        except:
            continue
    return valid_cards

def read_checkpoint() -> int:
    if os.path.exists(CHECKPOINT):
        try:
            with open(CHECKPOINT, "r", encoding="utf-8") as f:
                return max(1, int(f.read().strip()))
        except:
            return 1
    return 1

def write_checkpoint(page: int):
    with open(CHECKPOINT, "w", encoding="utf-8") as f:
        f.write(str(page))

def ensure_csv_header(path: str):
    if not os.path.exists(path) or os.path.getsize(path) == 0:
        with open(path, "w", encoding="utf-8-sig", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([
                "Tiêu đề dự án",
                "Giá",
                "Diện tích",
                "Giá/m²",
                "Địa chỉ",
                "Số phòng ngủ",
                "Số phòng vệ sinh",
                "Ngày đăng"
            ])

def main():
    driver = None
    try:
        driver = init_driver()
        wait = WebDriverWait(driver, WAIT_TIMEOUT)
        csv_path = os.path.join(os.getcwd(), OUTPUT_FILE)
        ensure_csv_header(csv_path)

        page = read_checkpoint()
        processed = 0

        while page <= SAFETY_CAP:
            url = build_url(page)
            print(f"==> Trang {page}: {url}")
            driver.get(url)

            # Cloudflare
            try:
                wait_cloudflare(driver, 60)
            except:
                try:
                    input("Nếu Cloudflare yêu cầu xác minh, giải xong rồi nhấn ENTER để tiếp tục...")
                except EOFError:
                    pass  # phòng trường hợp chạy môi trường không có stdin

            time.sleep(random.uniform(1.5, 2.5))
            scroll_to_bottom_stable(driver, pause=1.0, max_stable=2)

            cards = get_cards(driver, wait)
            if not cards:
                print("↳ Hết dữ liệu (không còn card hợp lệ). Dừng.")
                break

            print(f"↳ Số bài đăng hợp lệ: {len(cards)}")

            with open(csv_path, "a", encoding="utf-8-sig", newline="") as f:
                w = csv.writer(f)
                for c in cards:
                    row = extract_info(c)
                    if all(x == "N/A" for x in row):
                        continue
                    w.writerow(row)
                    processed += 1

            write_checkpoint(page)
            page += 1
            print(f"✅ Tổng đã ghi: {processed} dòng\n")
            time.sleep(random.uniform(1.6, 3.0))

        print(f"✅ Hoàn tất. Đã ghi {processed} dòng vào {csv_path}")

    except KeyboardInterrupt:
        print("\n⏸ Dừng bởi người dùng. Có thể chạy lại từ checkpoint.")
        # Không sys.exit() để tránh SystemExit trong Jupyter/VSCode
    finally:
        if driver:
            safe_quit(driver)

if __name__ == "__main__":
    main()


==> Trang 184: https://batdongsan.com.vn/nha-dat-ban-tp-hcm/p184
↳ Số bài đăng hợp lệ: 20
✅ Tổng đã ghi: 20 dòng

==> Trang 185: https://batdongsan.com.vn/nha-dat-ban-tp-hcm/p185
↳ Số bài đăng hợp lệ: 20
✅ Tổng đã ghi: 40 dòng

==> Trang 186: https://batdongsan.com.vn/nha-dat-ban-tp-hcm/p186
↳ Số bài đăng hợp lệ: 20
✅ Tổng đã ghi: 60 dòng

==> Trang 187: https://batdongsan.com.vn/nha-dat-ban-tp-hcm/p187
↳ Số bài đăng hợp lệ: 20
✅ Tổng đã ghi: 80 dòng

==> Trang 188: https://batdongsan.com.vn/nha-dat-ban-tp-hcm/p188
↳ Số bài đăng hợp lệ: 20
✅ Tổng đã ghi: 100 dòng

==> Trang 189: https://batdongsan.com.vn/nha-dat-ban-tp-hcm/p189
↳ Số bài đăng hợp lệ: 20
✅ Tổng đã ghi: 120 dòng

==> Trang 190: https://batdongsan.com.vn/nha-dat-ban-tp-hcm/p190
↳ Số bài đăng hợp lệ: 20
✅ Tổng đã ghi: 140 dòng

==> Trang 191: https://batdongsan.com.vn/nha-dat-ban-tp-hcm/p191
↳ Số bài đăng hợp lệ: 20
✅ Tổng đã ghi: 160 dòng

==> Trang 192: https://batdongsan.com.vn/nha-dat-ban-tp-hcm/p192
↳ Số bài đăng hợp l